In [11]:
import cv2
import mediapipe as mp 
import math

In [17]:
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
import numpy as np

In [26]:
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()

mpDraw = mp.solutions.drawing_utils

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_,CLSCTX_ALL,None)
volume = cast(interface,POINTER(IAudioEndpointVolume))
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 0
volPer = 0

while True:
    success,img = cap.read()

    imgRgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRgb)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id,lm in enumerate(handLms.landmark):
                h,w,c = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                x1,y1,x2,y2
                if id==4:
                    x1,y1 = cx,cy
                    cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
                if id==8:
                    x2,y2 = cx,cy
                    cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
                cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
                length = math.hypot(x2-x1,y2-y1)
                vol = np.interp(length,[25,120],[minVol,maxVol])
                volBar = np.interp(length,[25,120],[400,150])
                volPer = np.interp(length,[25,120],[0,100])

                volume.SetMasterVolumeLevel(vol,None)
            mpDraw.draw_landmarks(img,handLms,mpHands.HAND_CONNECTIONS)

    cv2.rectangle(img,(50,150),(85,400),(0,255,0),3)
    cv2.rectangle(img,(50,int(volBar)),(85,400),(0,255,0),cv2.FILLED)
    cv2.putText(img,f'{int(volPer)} %',(40,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)

    cv2.imshow("Image",img)
    if cv2.waitKey(1)==ord('q'):
        break

cv2.destroyAllWindows()
cap.release()